In [1]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import pandas as pd
import random
import json
import os
import math

In [2]:
BASE_DIR = f"/net/projects/scratch/winter/valid_until_31_July_2022/ybrima/data/genres/"

In [3]:
def get_files(BASE_DIR):
    CLASSES = os.listdir(BASE_DIR)
    ds =  {'file':[], "class":[]}
    for c in CLASSES:
        if(os.path.isdir(os.path.join(BASE_DIR,c))):
            files =  os.listdir(os.path.join(BASE_DIR,c))
            cur_dir = os.path.join(BASE_DIR,c)
            for f in files:
                file =  os.path.join(cur_dir,f)
                ds['file'].append(file)
                ds['class'].append(c)
    data = pd.DataFrame(ds)
    return data,CLASSES

In [4]:
df,classes = get_files(BASE_DIR)

In [5]:
df['class'].value_counts()

blues        100
jazz         100
reggae       100
hiphop       100
metal        100
classical    100
rock         100
pop          100
disco        100
country      100
Name: class, dtype: int64

In [7]:
file_path = "/net/projects/scratch/winter/valid_until_31_July_2022/ybrima/data/data_genre.npz"
SAMPLE_RATE = 44100
n_fft = 2048
hop_length =  512
num_mfcc=128

In [32]:
def save_array(df,file_path):
    X  = []
    W = []
    y =  []
    for k in range(df.shape[0]):
        r =  df.iloc[k]

        #   Loading the audio file 
        signal,_ = librosa.load(r['file'], sr=SAMPLE_RATE)
        # extract mfcc
        mfcc = librosa.feature.mfcc(signal, SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
        spec = librosa.amplitude_to_db(np.abs(librosa.stft(signal,n_fft=n_fft,hop_length=hop_length)), ref=np.max)
        w,h = 2586, 128
        mfcc = mfcc.T
        if(mfcc.shape[0] == w and mfcc.shape[1] == h):
            X.append(mfcc)
            W.append(spec)
            y.append(classes.index(r['class']))
    X =  np.array(X)
    W =  np.array(W)
    y =  np.array(y)
    data =  np.savez(file_path,w=W,x=X,y=y)
    print(f"Data written to storage successfully, path = {file_path}")

def plot_history(history):
    """Plots accuracy/loss for training/validation set as a function of the epochs
        :param history: Training history of model
        :return:
    """

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # create error sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.show()

def plot_sample(x,sr):
    
    if(x.shape[0] > x.shape[1]):
        x =  x.T
    librosa.display.specshow(x, sr=sr, x_axis='time',y_axis="log",hop_length=hop_length)
    plt.colorbar(format='%+2.f') 

In [8]:
df.to_csv('sanitized_music_genre', encoding='utf-8', index=False)

In [ ]:
save_array(df,file_path)

In [ ]:
data =  np.load(file_path)

In [ ]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import numpy as np
import scipy as sp
import pandas as pd
import random
import os
import math

In [ ]:
data = np.load(file_path)

In [ ]:
W,X,y = data['w'],data['x'],data['y']

In [ ]:
plot_sample(X[random.randint(0,len(X))],SAMPLE_RATE)

In [ ]:
# create train/test split
X_train, X_test, y_train, y_test = train_test_split(W, y, test_size=0.3,shuffle=True)

In [ ]:
# build network topology
model = keras.Sequential([

    # input layer
    keras.layers.Flatten(input_shape=(W.shape[1], W.shape[2])),

    # 1st dense layer
    keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.3),

    # 2nd dense layer
    keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.3),

    # 3rd dense layer
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.3),

    # output layer
    keras.layers.Dense(10, activation='softmax')
])

# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.0003)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
# train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=100)

# plot accuracy and error as a function of the epochs
plot_history(history)

In [ ]:
# plot accuracy and error as a function of the epochs
plot_history(history)